# Simple electricity market: Europe

## Single bidding zone with fixed load, one period

In this example we consider a single market bidding zone, Germany, France and Belgium.

The inelastic load has essentially infinite marginal utility (or higher than the marginal cost of any generator).

#### 1)Import Packages

In [2]:
import pypsa
import pandapower as pp
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import cartopy.crs as ccrs
import warnings
import subprocess
from shapely.errors import ShapelyDeprecationWarning
import logging

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("pypsa.pf").setLevel(logging.WARNING)
plt.rc("figure", figsize=(10, 8))

c:\Users\suhas\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


#### 2)Create a network and set Snapshots

In [3]:
# Create a new PyPSA network
network = pypsa.Network()
network.set_snapshots(range(1))  # Solve for a year 365*24

In [4]:
type(network)

pypsa.components.Network

In [5]:
os.chdir("C:\\Users\\par19744\\Python_projects\\practice notebooks\\PyPSA lectures BTU")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\par19744\\Python_projects\\practice notebooks\\PyPSA lectures BTU'

#### 3)Add Buses

In [ ]:
# Read excel file which contains Non renewable generators data
bus=pd.read_excel('data_Europe.xlsx',sheet_name='buses',header=0)

In [ ]:
bus

In [ ]:
for index, row in bus.iterrows():
    network.add(
        "Bus",
        name=row['bus'],
        v_nom=row['v_nom'],
        carrier=row['carrier'],
        x=row['x'],
        y=row['y']
    )

In [ ]:
network.buses

#### 4)Add generators

In [ ]:
# Read excel file which contains Non renewable generators data
generators=pd.read_excel('data_Europe.xlsx',sheet_name='generators')

In [ ]:
generators

In [ ]:
network.madd(
    "Generator",
    names= generators.generator,
    bus=list(generators.bus),
    carrier=list(generators.generator),
    p_nom=list(generators.p_nom),
    p_nom_max=list(generators.p_nom_max),  # MW
    p_nom_extendable=list(generators.p_nom_extendable),
    marginal_cost=list(generators.marginal_cost),
)
network.generators

In [ ]:
network.generators

NameError: name 'network' is not defined

#### 5)Add load

In [ ]:
# Read excel file which contains Non renewable generators data
load=pd.read_excel('data_Europe.xlsx',sheet_name='load',header=0)

In [ ]:
load

In [ ]:
for index, row in load.iterrows():
    network.add(
        "Load",
        name=row['bus'],
        bus=row['bus'],
        p_set=row['load'],
    )

In [ ]:
network.loads

#### 6)Add Transmission lines

In [ ]:
# Read excel file which contains Non renewable generators data
lines=pd.read_excel('data_Europe.xlsx',sheet_name='lines',header=0)

In [ ]:
lines.head()

In [ ]:
# Add line connection between the buses
for index, row in lines.iterrows():
    network.add(
        "Line",
        "{}_{}".format(row['country_1'],row['country_2']),
        bus0=row['country_1'],
        bus1=row['country_2'],
        s_nom=row['cap_MW'],
        x=1,
    )

In [ ]:
network.lines

In [ ]:
network

#### 7)Solve the network

In [ ]:
network.lopf(solver_name='glpk')

#### 8)Visualization of results

In [ ]:
network.buses_t.marginal_price

In [ ]:
clearing_price_redisp = network.buses_t.marginal_price.iloc[0].max()
clearing_price_redisp

In [ ]:
network.generators.p_nom_opt

In [ ]:
network.generators.p_nom_opt.plot.bar(ylabel="MW", figsize=(8, 3))

In [ ]:
network.lines_t.p0

In [ ]:
gen = network.generators.assign(g=network.generators_t.p.mean()).groupby(["bus", "carrier"]).g.sum()
gen

In [ ]:
flow = pd.Series(10, index=network.branches().index)
flow

In [ ]:
plot_graph=network.plot(
    bus_sizes=gen/10,
    bus_colors={"Germany Wind":"midnightblue","Germany Coal": "black", "Germany Gas": "gray","France Gas":"gray","France Wind":"midnightblue","France Hydro":"cyan","Belgium Solar":"yellow","Belgium Gas":"gray"},
    margin=1,
    line_widths=3,
    link_widths=0,
    flow=flow,
    color_geomap=True,
    projection=ccrs.EqualEarth(),
    line_colors=network.lines_t.p0.mean().abs(),
)
plt.colorbar(plot_graph[2], fraction=0.04, pad=0.004, label="Flow in MW")
plt.show()